<a href="https://colab.research.google.com/github/kevincong95/cs231n-emotiw/blob/master/notebooks/2.0-la-tj-ak-ensemble_baseline_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Video Sentiment Analysis in the Wild
### Ensembling Notebook | CS231n

This notebook preprocesses input videos to extract faces, frames, poses, and audio before running pre-trained models for each modality to predict group sentiment (positive, negative, or neutral). 

In [2]:

# Clone the code base
!git clone 'https://github.com/kevincong95/cs231n-emotiw.git'

# Switch to TF 1.x and navigate to the directory
%tensorflow_version 1.x
!pwd
import os
os.chdir('cs231n-emotiw')
!pwd

# Install required packages 
!pip install -r 'requirements-predictions.txt'


fatal: destination path 'cs231n-emotiw' already exists and is not an empty directory.
TensorFlow 1.x selected.
/content
/content/cs231n-emotiw


#### Pose Pre-Requisites
Pose extraction uses the [CMU OpenPose library](https://github.com/CMU-Perceptual-Computing-Lab/openpose) to extract body keypoints. We have pre-compiled this library for use in Colab but some system files still need to be installed. 

#### Retrieve the files

The code block below demonstrates how to retrieve the files from GCS. However, feel free to skip this step if the files are already on the local disk or you have Google Drive mounted.

In [3]:
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
!wget https://storage.googleapis.com/cs231n-emotiw/openpose/openpose.tar.gz
!tar -xzf openpose.tar.gz

!wget https://storage.googleapis.com/cs231n-emotiw/data/train-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/val-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/test-tiny.zip
!wget https://storage.googleapis.com/cs231n-emotiw/data/Train_labels.txt
!wget https://storage.googleapis.com/cs231n-emotiw/data/Val_labels.txt


--2020-05-30 08:35:14--  https://storage.googleapis.com/cs231n-emotiw/openpose/openpose.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.206.128, 2a00:1450:400c:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.206.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 598708936 (571M) [application/x-tar]
Saving to: ‘openpose.tar.gz.2’

openpose.tar.gz.2   100%[===================>] 570.97M  91.0MB/s    in 6.1s    

2020-05-30 08:35:20 (93.7 MB/s) - ‘openpose.tar.gz.2’ saved [598708936/598708936]

--2020-05-30 08:35:31--  https://storage.googleapis.com/cs231n-emotiw/data/train-tiny.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.184.128, 2a00:1450:400c:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.184.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50131084 (48M) [application/zip]
Saving to: ‘train-tiny.zip.2’

train-tiny.zip.2

#### Preprocess Files

Here, we will instantiate each of the preprocessors and process all of the input video files.

NOTE: Change the input parameters as needed.

WARNING: This may take several hours to complete, depending on the number of files.

In general, pre-processing will extract the following:
- Video frames
- Pose keypoints
- Faces from each video frame
- Audio waveform and audio features

In [4]:
from src.preprocessors.preprocess_all_modes import preprocess
from src.preprocessors.pose_preprocessor import PosePreprocessor

print("Starting to preprocess train data")
preprocess(video_folder="train-tiny.zip", label_file="Train_labels.txt", local_base_path="train-tiny")

# print("Starting to preprocess val data")
# preprocess(video_folder="val-tiny.zip", label_file="Val_labels.txt", local_base_path="val-tiny")

# print("Starting to preprocess test data")
# preprocess(video_folder="test-tiny.zip", local_base_path="test-tiny")

Using TensorFlow backend.


Starting to preprocess train data
Video Preprocessor created with video_folder = train-tiny.zip , label_file = Train_labels.txt , output_folder = train-tiny-frames, output_file = train-tiny-frames.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Video Preprocessor created with video_folder = train-tiny.zip , output_folder = train-tiny-faces, output_file = train-tiny-faces.zip
Frames will be created with height = 320 , width = 480 , sample_every = 10
Pose Preprocessor created with is_test = False, video_frame_folder = train-tiny-frames , output_folder = train-tiny-pose, output_file = train-tiny-pose.zip
Video Preprocessor created with video_folder = train-tiny.zip , output_folder = train-tiny-audio, output_file = train-tiny-audio.zip
Frames will be created with hop_size = 0.5
Unzipping files to temp dir train-tiny-frames_tmp...
Finished unzipping files
Found 50 videos
Processing video 1/50 with name 101_12.mp4 and class 3 

Processing video 2/50 with name 1

In [0]:
# Remove the openpose folder as it is no longer required
!rm -rf openpose/

### Run Classifiers

In [0]:
%tensorflow_version 2.x

In [2]:
import tensorflow
print(tensorflow.__version__)

2.2.0


In [2]:
!pwd
import os
os.chdir('cs231n-emotiw')
!pwd


/content
/content/cs231n-emotiw


In [3]:
!git pull

remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 10 (delta 8), reused 10 (delta 8), pack-reused 0
Unpacking objects: 100% (10/10), done.
From https://github.com/kevincong95/cs231n-emotiw
   b05e030..be272a2  master     -> origin/master
Updating b05e030..be272a2
Fast-forward
 .DS_Store                                          |  Bin 8196 -> 10244 bytes
 ....0-la-tj-ak-ensemble_baseline_predictions.ipynb | 1331 --------------------
 notebooks/image_caption/.DS_Store                  |  Bin 6148 -> 6148 bytes
 src/classifiers/audio_classifier.py                |    2 +-
 src/classifiers/frames_classifier.py               |    4 +-
 src/classifiers/pose_classifier.py                 |    2 +-
 6 files changed, 4 insertions(+), 1335 deletions(-)
 delete mode 100644 notebooks/2.0-la-tj-ak-ensemble_baseline_predictions.ipynb


In [12]:
!ls -l train-tiny-audio

total 4
drwxr-xr-x 2 root root 4096 May 30 08:18 audio-pickle


In [4]:
from src.classifiers.audio_classifier import AudioClassifier
from src.classifiers.frames_classifier import FramesClassifier
from src.classifiers.pose_classifier import PoseClassifier
from src.classifiers.utils import get_num_samples
import numpy as np

audio_classifier = AudioClassifier('train-tiny-audio', model_location='https://storage.googleapis.com/cs231n-emotiw/models/OPENL3_audio_api_train_test-1-500-epochs-0.5_hop--BEST_MODEL-w-VAL--1e-6-lr-0.2-dropout-512-feat-map-batch-norm-3-cnn-layers.h5', is_test=False)
frames_classifier = FramesClassifier('train-tiny-frames', model_location='https://storage.googleapis.com/cs231n-emotiw/models/frame-classifier-resnet-lstm-x3.h5', is_test=False)

# TODO: Does not currently work due to problem with OpenPose feature extraction
# pose_classifier = PoseClassifier('train-tiny-pose', model_location='https://storage.googleapis.com/cs231n-emotiw/models/pose-classifier-v5.h5', is_test=False)

classifiers = [audio_classifier, frames_classifier]

#
# MAX CLASSIFIER
#
classifier_outputs = []
classifier_samples = []
sample_to_row = {}
num_samples = 0

for c, classifier in enumerate(classifiers):
    results, samples = classifier.predict()
    classifier_outputs.append(results.tolist())
    classifier_samples.append(list(samples))
    num_samples = len(list(samples))
    print(num_samples)

for i, sample in enumerate(classifier_samples[0]):
    sample_to_row[sample] = i

X = np.zeros(shape=(num_samples, len(classifiers) * 3))
for c, output in enumerate(classifier_outputs):
    samples = classifier_samples[c]
    for i, row in enumerate(output):
        sample = samples[i]
        X[sample_to_row[sample], np.argmax(row).item()] += 1

print(np.argmax(X, axis=1))


AudioClassifier created with audio_folder = train-tiny-audio , is_test = False , model_location = https://storage.googleapis.com/cs231n-emotiw/models/OPENL3_audio_api_train_test-1-500-epochs-0.5_hop--BEST_MODEL-w-VAL--1e-6-lr-0.2-dropout-512-feat-map-batch-norm-3-cnn-layers.h5
FramesClassifier created with frames_folder = train-tiny-frames , is_test = False , model_location = https://storage.googleapis.com/cs231n-emotiw/models/frame-classifier-resnet-lstm-x3.h5
Skipping unzipping files as input is a folder
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument nu

In [0]:
def predict(mp4_dir , train_target_path , model_list= [model,] , model_paths=["",] , num_models = 1 , mode="soft" , complexFusion=False):

  """

  Notes: 

  All file preprocessing will occur in this function.


  Inputs
  
  * mp4_train_dir - The directory of .mp4 file paths that the model will make predictions from.
  * train_target_path - The target path. Should be in .txt format.
  * A list of pretrained models that will be used in the ensemble during prediction


  Outputs 

  * An array of size (M) where M is the number of samples

  """

  assert len(model_list) == num_models
  assert len(model_paths) == num_models

  all_model_predictions = []
  X_list = []

  counter = 0
  
                            

  for model in models:

    model.load_model(model_paths[counter])


    X = X_list[counter]
    
   

    all_model_predictions.append(model.predict(dir="video path/*.mp4" , X=X)) # Predict returns an (M , 3) array
    
    counter += 1

  all_model_predictions = np.asarray(all_model_predictions , dtype='float32') # (num_models , M , 3)


  assert mode in ["soft" , "hard"]

  if mode == "soft":

    
    # Take the average of each 

    predictions = np.mean(all_model_predictions , axis=0)

    predictions = np.argmax(predictions , axis = 1)

    return predictions  # (M,) in the domain [0,1,2]

  # TODO: Majority vote

  positive_hard = np.argmax(positive_arr , axis=1) # ()



  return predictions 

In [0]:
def evaluate(predictions, targets):

  # TO DO: Add evaluate code here

  """
  Inputs
  
  * predictions np array of shape M where M is the number of samples
  * target array of shape M where M is the number of samples


  Outputs 

  * Model accuracy scalar 

  """

  assert len(targets) == len(predictions)

  incorrect = np.count_nonzero(predictions - targets)

  acc = (targets.shape[0] - incorrect) / targets.shape[0]

  return acc


